### RootInteractive for comparison of 2015/2018 and MC 2018 anchor
Juputer notebook tutorial/troubleshooting macro to visualize TPC performance and to create dynamic dashborads (standalone html files create in specified location)

!!! IMPORTENT - TO NOTE - performance maps used in this example has a high granularity It is impossible to borwse all point interactivelly using bokeh client Depending on the browser setting - slowdonw is obdserved at ~ 50000 -100000 points

* Load libraries
* Load perormance trees for different periods
* Define alias tree variables used in furtehr visualization
* Make perfromance comparison dashboards

In [ ]:
from RootInteractive.InteractiveDrawing.bokeh.bokehDrawSA import *
from ROOT import TFile, AliTreePlayer, TStatToolkit,TCanvas
from RootInteractive.Tools.aliTreePlayer import *
from bokeh.io import curdoc,output_notebook
output_notebook()
import warnings
from pprint import pprint
from RootInteractive.MLpipeline.NDFunctionInterface import  DataContainer, Fitter

In [ ]:
ROOT.TFile.SetCacheFileDir("../data/")
treeSec, treeList, fileList = LoadTrees("cat map.list",".*_dSec.*","(.*proj.*|.*1Dist|.*2Dist)",".*",0)
#treeSec = AliTreePlayer.LoadTrees("cat map.list",".*_dSec.*","(.*proj.*|.*1Dist|.*2Dist)",".*","","")
treeSec.SetAlias("qPt","qPtCenter")
treeSec.SetAlias("dSector","dSectorCenter")
treeSec.SetAlias("atgl","atglCenter")
treeSec.SetAlias("multTPCClusterN","multTPCClusterNCenter")
treeSec.SetAlias("selected","LHC18l8a.hisnormChi2TPC_dSecDist.entries>50&&rndm<0.5")
AddMetadata(treeSec,"dSector.AxisTitle","x_sector (a.u.)")
AddMetadata(treeSec,"qPt.AxisTitle","x_sector (a.u.)")
AddMetadata(treeSec,"atgl.AxisTitle","pz/pt")
AddMetadata(treeSec,"multTPCClusterN.AxisTitle","Ncl/Ncl_central")
treeSec.Draw(">>selectedList","selected","entrylist")
treeSec.SetEntryList(ROOT.gROOT.FindObject("selectedList"))

## Browse the content of the Tree DB (set of trees)
* enable line below to get tree content

In [ ]:
#print(RenderTree(treeSec.anyTree))
# findSelectedBranches(treeSec,[".*Center.*"],exclude=[".*LHC.*",".*log.*"], columnMask=[["Center",""]])
#findSelectedBranches(treeSec,[".*LHC15o.*hi2.*"],exclude=[".*log.*"], columnMask=[["Center",""]])
#hisncrROCA_
findSelectedBranches(treeSec,[".*LHC15o.*ncrROCA.*"],exclude=[".*log.*"], columnMask=[["Center",""]])

# Select variables of interest
*  local position - delta Sector
*  pz/pt -atgl
*  Q/pt  
* TPC multiplicity norm. to central events

In [ ]:
varMedian=[".*Center"]
#print(findSelectedBranches(treeSec,varMedian,exclude=[".*LHC.*",".*log.*"], columnMask=[["Center",""]]))
dfVariables=tree2Panda(treeSec,[".*Center.*"],"entries>=0",exclude=[".*LHC.*",".*log.*",".*deltaP0.*"], columnMask=[["Center",""]])
varList=dfVariables.columns.values
#print("Variable list:", varList)
dfVariables.head(2)
varList2=[a for a in dfVariables.columns.values]
varList2.remove("dSector")
pprint(list(varList))

# Prepare the variable of interest to fit and visualize
* In example below it is combined dEdx and IROC dEdx  for MIP particles
* Random forest regression is used excluding deltaSector variable

In [ ]:
findSelectedBranches(treeSec,[".*LHC15o.*Chi2ITS.*binMedian.*"],exclude=[".*log.*"], columnMask=[["Center",""]])

In [ ]:
dfTPCChi2N=tree2Panda(treeSec,[".*Chi2TPC.*entries"],"entries>=0",exclude=[".*entriesG.*"],columnMask=[["_hisnormChi2TPC_dSecDist_entries","Chi2TPCN"]])
dfTPCChi2NList=list(dfTPCChi2N.columns.values)
dfITSChi2N=tree2Panda(treeSec,[".*Chi2ITS.*entries"],"entries>=0",exclude=[".*entriesG.*"],columnMask=[["_hisnormChi2ITS_dSecDist_entries","Chi2ITSN"]])
dfITSChi2NList=list(dfITSChi2N.columns.values)
dfTPCChi2=tree2Panda(treeSec,[".*Chi2TPC.*binMedian"],"entries>=0",columnMask=[["_hisnormChi2TPC_dSecDist_binMedian","Chi2TPC"]])
dfTPCChi2List=list(dfTPCChi2.columns.values)
dfITSChi2=tree2Panda(treeSec,[".*Chi2ITS.*binMedian"],"entries>=0",columnMask=[["_hisnormChi2ITS_dSecDist_binMedian","Chi2ITS"]])
dfITSChi2List=list(dfITSChi2.columns.values)
#
#hisncrROCA_dSecDist
dfTPCNcr=tree2Panda(treeSec,[".*ncrROCA.*mean0"],"entries>=0",exclude=[".*meanG.*"],columnMask=[["_hisncrROCA_dSecDist_mean","Ncr"]])
dfTPCNcrList=list(dfTPCNcr.columns.values)


#dfdEdx=pd.concat([dfdEdx, dfVariables],axis=1)
#print(dfdEdxList)
#dfdEdx.head(5)
dfStat=tree2Panda(treeSec,[".*hisdEdxMIP_dSecDist.*entries"],"entries>=0",exclude=[".*entriesG.*"],columnMask=[["_hisdEdxMIP_dSecDist_entries","N"]])
for a in dfStat.columns.values:
    dfStat[a+"S"]=1/np.sqrt(1+dfStat[a])
#print(dfStat.columns.values)  

In [ ]:
dfdEdxList=dfTPCChi2NList+dfTPCChi2List+dfITSChi2List+dfTPCNcrList

In [ ]:
df=pd.concat([dfVariables, dfTPCChi2N, dfITSChi2N,dfTPCChi2,dfITSChi2, dfTPCNcr],axis=1)
npoints=df.shape[0]

for varFit in dfTPCChi2NList+dfITSChi2NList+dfTPCChi2List+dfITSChi2List+dfTPCNcrList:
    dataContainer = DataContainer(df,  varList2, varFit, [npoints//2,npoints//2]) 
    fitter = Fitter(dataContainer)
    fitter.Register_Method('RF','RandomForest', 'Regressor', n_estimators=100, max_depth=10)
    #fitter.Register_Method('RF200','RandomForest', 'Regressor', n_estimators=200, max_depth=10)
    fitter.Fit()
    fitter.AppendStatPandas("RF",df,varFit)
    #fitter.AppendStatPandas("RF200",df,varFit)
    #df[varFit+'RFPull']=(df[varFit]-df[varFit+"RFMean"])/df[varFit+"RFRMS"]

In [ ]:
df.head(5)

In [ ]:
tooltips=[ (a,"@"+a) for a in varList ]
tooltips+=[ (a,"@"+a) for a in dfdEdxList]
figureArray=[ [["dSector"],[a+"/"+a+"RFMedian"],{"colorZvar":"qPt","size":3}] for a in dfTPCChi2NList ]
figureArray+=[ [["dSector"],[a+"/"+a+"RFMedian"],{"colorZvar":"qPt","size":3}] for a in dfITSChi2NList ]
figureArray+=[ [["dSector"],[a+""],{"colorZvar":"qPt","size":3}] for a in dfTPCChi2List ]
figureArray+=[ [["dSector"],[a+""],{"colorZvar":"qPt","size":3}] for a in dfITSChi2List ]           
figureArray+=[["table",{'include':'LHC15o'}]]

#
wIndex=0
widgets="query.custom(),"
widgetList=[
    ['slider', ['multTPCClusterN'],{'type':'unique'}],
    ['range', ['dSector']],
    ['slider', ['atgl'],{'type':'unique'}],
    ['range', ['qPt']]
]
#
widgetLayout=[]
index=0
for a in dfStat.columns.values: 
    if "entriesS" in a:
        widgetList.append(['range',[a],{'bins':30}])
#pprint(figureArray)        
#pprint(widgetList)
figureLayout=[
    [0,1,2,3, {'commonX':1,'commonY':1,'y_visible':2}],
    [4,5,6,7, {'commonX':1,'commonY':4,'y_visible':2}],
    #[8,9,10,11,{'commonX':1,'commonY':8,'y_visible':2}],
    #[12,{'plot_height':50}],
    {'plot_height':400,'sizing_mode':'scale_width','legend_visible':False}
]
widgetLayout=[ [0,1], [2,3], {'sizing_mode':'scale_width'} ]
output_file("compararisondEdxMC.html") 
fig=bokehDrawSA.fromArray(df, "abs(qPt)<2", figureArray,widgetList,tooltips=tooltips, layout=figureLayout,widgetLayout=widgetLayout,sizing_mode="scale_width") 

In [ ]:
tooltips=[ (a,"@"+a) for a in varList ]
tooltips+=[ (a,"@"+a) for a in dfdEdxList]
figureArray=[ [["dSector"],[a+""],{"colorZvar":"qPt","size":3}] for a in dfTPCNcr ]
figureArray+=[ [["dSector"],["("+a+"/"+a+"RFMedian)/(LHC18l8aChi2ITSN/LHC18l8aChi2ITSNRFMedian)"],{"colorZvar":"qPt","size":3}] for a in dfITSChi2NList ]
figureArray+=[ [["dSector"],[a+""],{"colorZvar":"qPt","size":3}] for a in dfTPCChi2List ]
figureArray+=[ [["dSector"],[a+""],{"colorZvar":"qPt","size":3}] for a in dfITSChi2List ]           
figureArray+=[["table",{'include':'LHC15o'}]]

#
wIndex=0
widgets="query.custom(),"
widgetList=[
    ['slider', ['multTPCClusterN'],{'type':'unique'}],
    ['range', ['dSector']],
    ['slider', ['atgl'],{'type':'unique'}],
    ['range', ['qPt']]
]
#
widgetLayout=[]
index=0
for a in dfStat.columns.values: 
    if "entriesS" in a:
        widgetList.append(['range',[a],{'bins':30}])
#pprint(figureArray)        
#pprint(widgetList)
figureLayout=[
    [0,1,2,3, {'commonX':1,'commonY':1,'y_visible':2}],
    [4,5,6,7, {'commonX':1,'commonY':4,'y_visible':2}],
    {'plot_height':400,'sizing_mode':'scale_width','legend_visible':False}
]
widgetLayout=[ [0,1], [2,3], {'sizing_mode':'scale_width'} ]
output_file("compararisondEdxMC.html") 
fig=bokehDrawSA.fromArray(df, "abs(qPt)<1", figureArray,widgetList,tooltips=tooltips, layout=figureLayout,widgetLayout=widgetLayout,sizing_mode="scale_width") 

In [ ]:
treeSec.Draw("LHC15o.hisncrROCA_dSecDist.binMedian","","")
ROOT.c1.Draw()